In [1]:
import re
import numpy as np

!git clone https://github.com/IlyaSk10/stress_symbol_predict/
%cd stress_symbol_predict
!unzip train_dataset.zip

# чтение словаря
with open("All_Forms_1.txt",'r',encoding='UTF-8') as f:
  file=f.readlines()

# заменить любые символы до # включительно символом "", затем найти все символы кроме "," 
ret=[re.findall(r'[^,]+',re.sub(r'\w+#','',i)) for i in file]   



CHARS = ['-', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к',
         'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш',
         'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё']
CHAR_INDICES = dict((c, i+1) for i, c in enumerate(CHARS))


# создание структуры "слово:номер буквы на которую падает ударение"
df={}
list2=[]
words_without_stress=[]
for string in ret:
    for word in string:
        if len(re.findall(r'(.+)\'|(.+)\`',word.strip()))>0: # проверить есть ли в слове знак ударения ' или `
            #front=re.findall(r'(.+)\'|(.+)\`',word.strip())
            front=re.sub(r'[\`\']','',re.findall(r'(.+)[\`\']',word.strip())[0]) # определить символы до знаков ударения
            clean_word=''.join(re.findall(r'[^\'\`]',word.strip())) # очистить слово от знаков ударения
            #df.update({clean_word:len(max(*front))})
            df.update({clean_word:len(front)})
    if len(df)>0:
      list2.append(df.copy())
      df.clear()
    else:
      words_without_stress.append(string)
  
print('Слова без разметки',words_without_stress[:20]) # слова без разметки и поэтому не учитываются далее

#MAXLEN=max(len(word) for k in range(len(list2)) for word in list2[k].keys())
#print('Максимальная длина слова с разметкой', MAXLEN, 'символов')

Cloning into 'stress_symbol_predict'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 0), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.
/content/stress_symbol_predict
Archive:  train_dataset.zip
  inflating: All_Forms_1.txt         
Слова без разметки [['\ufeffа\n'], ['абы\n'], ['авансом\n'], ['ага\n'], ['агами\n'], ['агу\n'], ['адажио\n'], ['адски\n'], ['аж\n'], ['азу\n'], ['актёр', 'актёры', 'актёра', 'актёров', 'актёру', 'актёрам', 'актёра', 'актёров', 'актёром', 'актёрами', 'актёре', 'актёрах\n'], ['актёришка', 'актёришки', 'актёришки', 'актёришек', 'актёришке', 'актёришкам', 'актёришку', 'актёришек', 'актёришкой', 'актёришкою', 'актёришками', 'актёришке', 'актёришках\n'], ['актёрка', 'актёрки', 'актёрки', 'актёрок', 'актёрке', 'актёркам', 'актёрку', 'актёрок', 'актёркой', 'актёркою', 'актёрками', 'актёрке', 'актёрках\n'], ['актёрство', 'актёрства

In [6]:
print("Слово максимальной длины: ",max(sum(ret,[]),key=len))

Слово максимальной длины:  народно-наро`дно-демократи'ческий


In [0]:
# функция преобразования каждого слова в набор чисел
def preparation (list2):
  total=np.sum([len(list2[i]) for i in range(len(list2))])
  A=np.zeros(shape=(total,MAXLEN))
  B=np.zeros(shape=(total,1))
  m=0
  for k in range(len(list2)):
    for i in range(len(list2[k])):
        x = np.zeros((1, MAXLEN)) 
        word=list(list2[k].keys())[i]
        for index, letter in enumerate(word.strip()):
            x[0,index]=CHAR_INDICES[letter] # слово представлено вектором чисел от 0 до 33
        A[m,:]=x
        B[m,:]=list2[k][word]
        m=m+1
  return A,B

In [0]:
import random
list_number_elements=20000
random.shuffle(list2) # перемешиваем данные
X_test,y_test=preparation(list2[:list_number_elements]) # и берем первые 20000 штук элементов списка. Каждый элемент списка это (слово+все словообразования)
X_train,y_train=preparation(list2[list_number_elements:])

from keras.utils.np_utils import to_categorical
y_test_cat = to_categorical(y_test, num_classes=MAXLEN)
y_train_cat = to_categorical(y_train, num_classes=MAXLEN)

In [32]:
#функция конвертирует вектор в слово
def convert(X): 
  word_concat=[]
  for word in X:
    word_concat.append(''.join([key for index,number in enumerate(word) for key,value in CHAR_INDICES.items() if value==number]))
  return word_concat

w=list((key,value) for i in range(len(list2)) for key,value in list2[i].items())
number=np.sum([len(list2[i]) for i in range(len(list2[:list_number_elements]))])

number_of_words=1000
word_concat=convert(X_train[0:number_of_words])

for i in range(0,len(y_train_cat[:number_of_words]),20):
  lab=np.argmax(y_train_cat[i])
  print('Слово после конвертации :',word_concat[i][:lab]+"'"+word_concat[i][lab:],'|','Слово из словаря :',w[number+i][0][:w[number+i][1]]+"'"+w[number+i][0][w[number+i][1]:])

Слово после конвертации : уточи'ться | Слово из словаря : уточи'ться
Слово после конвертации : уточи'вшемуся | Слово из словаря : уточи'вшемуся
Слово после конвертации : широкоро'того | Слово из словаря : широкоро'того
Слово после конвертации : мягкосерде'чию | Слово из словаря : мягкосерде'чию
Слово после конвертации : ко'м | Слово из словаря : ко'м
Слово после конвертации : упраздня'ющейся | Слово из словаря : упраздня'ющейся
Слово после конвертации : упраздня'вшемся | Слово из словаря : упраздня'вшемся
Слово после конвертации : безынтере'сную | Слово из словаря : безынтере'сную
Слово после конвертации : ко'шере | Слово из словаря : ко'шере
Слово после конвертации : побра'жничавшего | Слово из словаря : побра'жничавшего
Слово после конвертации : зуболече'бное | Слово из словаря : зуболече'бное
Слово после конвертации : растениево'дства | Слово из словаря : растениево'дства
Слово после конвертации : побушу'ем | Слово из словаря : побушу'ем
Слово после конвертации : побушева'вшим | Сло

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Embedding,Bidirectional,LSTM
from keras.utils.vis_utils import plot_model
# создание сети
model = Sequential()
vocab_size=len(CHARS)
model.add(Embedding(vocab_size, output_dim= len(CHARS),input_length=MAXLEN))
model.add(Bidirectional(LSTM(units=128, return_sequences=True, input_shape=(MAXLEN, len(CHARS)),dropout = 0.3)))
model.add(Bidirectional(LSTM(units=128,dropout = 0.3)))
model.add(Dense(MAXLEN, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
#model.summary()

# обучение модели
model.fit(X_train, y_train_cat, batch_size = 64,epochs=1,validation_data = (X_test, y_test_cat),verbose = 1)

In [0]:
from sklearn.metrics import accuracy_score

def accuracy(X_test):
  predicted=model.predict(X_test)
  predicted = np.argmax(predicted, axis=1)
  return accuracy_score(y_test, predicted)

print('Доля правильных ответов','{:.3f}'.format(accuracy(X_test)))

Доля правильных ответов 0.905


In [24]:
number_of_words=3000 # возьмем первые 3000 слов из test
word_concat_test=convert(X_test[:number_of_words])

correct_predict=[] # список для слов в которых верно проставлено ударение
incorrect_predict=[] # список для слов в которых неверно проставлено ударение

for i in range(0,len(X_test[:number_of_words]),50):
  predicted=model.predict(X_test[i].reshape(1,MAXLEN))
  predicted = np.argmax(predicted, axis=1)
  if y_test[i]==predicted:
    correct_predict.append(word_concat_test[i][:int(predicted)]+"'"+word_concat_test[i][int(predicted):])
  else:
    incorrect_predict.append(word_concat_test[i][:int(predicted)]+"'"+word_concat_test[i][int(predicted):])

print([('Правильный ответ :',word_correct) for word_correct in correct_predict])
print([('Неправильный ответ :',word_incorrect) for word_incorrect in incorrect_predict])

[('Правильный ответ :', "фарисе'йскими"), ('Правильный ответ :', "нормализова'вшемуся"), ('Правильный ответ :', "подкра'шиванием"), ('Правильный ответ :', "задрапирова'вшейся"), ('Правильный ответ :', "захлестну'сь"), ('Правильный ответ :', "концептуали'зм"), ('Правильный ответ :', "вмя'лись"), ('Правильный ответ :', "услажда'ющиеся"), ('Правильный ответ :', "надключи'чнее"), ('Правильный ответ :', "мы'лен"), ('Правильный ответ :', "обуржуа'зено"), ('Правильный ответ :', "десятигра'нниках"), ('Правильный ответ :', "настрое'нием"), ('Правильный ответ :', "закру'чиваний"), ('Правильный ответ :', "погребны'е"), ('Правильный ответ :', "обеззву'ча"), ('Правильный ответ :', "соку'рснице"), ('Правильный ответ :', "попа'ханных"), ('Правильный ответ :', "де'вственное"), ('Правильный ответ :', "перестре'лянное"), ('Правильный ответ :', "припе'рченную"), ('Правильный ответ :', "ледови'тое"), ('Правильный ответ :', "разде'рганными"), ('Правильный ответ :', "благоприли'чиями"), ('Правильный ответ :

In [0]:
path='/content/drive'
# сохранение модели и весов
model_json = model.to_json()
with open(path+"model_6.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights(path+"model_6.h5")
print("Saved model to disk")

Saved model to disk
